# Advertiser Status

You're provided with two tables: the `advertiser` table contains information about advertisers & their respective payment status, & the `daily_pay` table contains the current pay information for advertisers, & it only includes advertisers who have made payments.

Write a query to update the payment status of Facebook advertisers based on the information in the `daily_pay` table. The output should include the user ID & their current payment status, sorted by the user id.

The payment status of advertisers can be classified into the following categories:

* New: Advertisers who are newly registered & have made their first payment.
* Existing: Advertisers who have made payments in the past & have recently made a current payment.
* Churn: Advertisers who have made payments in the past but not have made any recent payment.
* Resurrect. Advertisers who have not made a recent payment but may have made a previous payment & have made a payment again recently.

Before proceeding with the prompt, it is important to understand the possible transitions in the advertiser's status based on the payment status. The following table provides a summary of these transitions:

|#|Current Status|Updated Status|Payment on Day T|
|:---|:---|:---|:---|
|1|NEW|EXISTING|Paid|
|2|NEW|CHURN|Not paid|
|3|EXISTING|EXISTING|Paid|
|4|EXISTING|CHURN|Not paid|
|5|CHURN|RESURRECT|Paid|
|6|CHURN|CHURN|Not paid|
|7|RESURRECT|EXISTING|Paid|
|8|RESURRECT|CHURN|Not paid|

* "Current Status" column: Represents the advertiser's current status
* "Payment Status" column: Represents the updated payment status based on the conditions
* "Payment on Day T" column: Indicates whether the advertiser made a payment on a specific day (T) or not

The transitions between payment statuses in the provided table can be summarised as follows:

* Rows 2, 4, 6, & 8: If an advertiser does not make a payment on day T, regardless of their previous status, their payment status transitions to "CHURN" as the updated status.
* Rows 1, 3, 5, & 7: If an advertiser makes a payment on day T, the status is updated to either "EXISTING" or "RESURRECT" based on their previous status. If the previous status was "CHURN", the updated status is "RESURRECT". For any other previous status, the updated status is "EXISTING".

# Answer

The `advertiser` table:

```
CREATE TABLE advertiser (
	user_id varchar(10),
	status varchar(10)
);

INSERT INTO advertiser
VALUES ('bing', 'NEW'),
	   ('yahoo', 'NEW'),
	   ('alibaba', 'EXISTING'),
	   ('baidu', 'EXISTING'),
	   ('target', 'CHURN'),
	   ('tesla', 'CHURN'),
	   ('morgan', 'RESURRECT'),
	   ('chase', 'RESURRECT');

SELECT * FROM advertiser;
```

<img src = "advertiser Table.png" width = "600" style = "margin:auto"/>

The `daily_pay` table:

```
CREATE TABLE daily_pay (
	user_id varchar(10),
	paid numeric(5, 2)
);

INSERT INTO daily_pay
VALUES ('yahoo', 45.00),
	   ('alibaba', 100.00),
	   ('target', 13.00),
	   ('morgan', 600.00),
	   ('fitdata', 25.00);

SELECT * FROM daily_pay;
```

<img src = "daily_pay Table.png" width = "600" style = "margin:auto"/>

This is a job for `CASE WHEN`.

```
SELECT *,
	   (CASE
	   	   WHEN advertiser.status IN ('NEW', 'EXISTING',
			   'RESURRECT') AND daily_pay.paid IS NOT NULL
			   THEN 'EXISTING'
		   WHEN advertiser.status = 'CHURN' AND
		   	   daily_pay.paid IS NOT NULL THEN 'RESURRECT'
	    ELSE 'CHURN'
		END) AS updated_status
FROM advertiser
LEFT JOIN daily_pay USING (user_id);
```

<img src = "Updated Status.png" width = "600" style = "margin:auto"/>

Yahoo, Alibaba & Morgan have an updated status of "EXISTING", while Target has an updated status of "RESURRECT". The rest of the companies have an updated status of "CHURN", because they haven't made their payment yet.